In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd

In [2]:
# Plank dataset class
class PlankDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        features = torch.tensor(self.data.iloc[idx].values, dtype=torch.float32)
        return features

In [3]:
# Plank CNN model

class PlankCNN(nn.Module):
    def __init__(self):
        super(PlankCNN, self).__init__()
        self.conv1 = nn.Conv1d(1, 8, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(8, 16, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool1d(2)
        self.fc1 = nn.Linear(16, 7)  # Corrected input size to match the number of features 

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = x.view(-1, 16)  # Corrected flattening
        x = self.fc1(x)
        return x

In [4]:
# data split and loader
dataset = PlankDataset('../../../Computer_Vision/plank_features.csv')
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [5]:
model = PlankCNN()
optimizer = optim.Adam(model.parameters())
criterion = nn.MSELoss()

In [6]:
num_epochs = 10000
for epoch in range(num_epochs):
    # Training
    model.train()
    train_epoch_loss = 0.0
    for i, data in enumerate(train_dataloader):
        inputs = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()
        train_epoch_loss += loss.item()

    # Validation
    model.eval()
    val_epoch_loss = 0.0
    with torch.no_grad():
        for i, data in enumerate(val_dataloader):
            inputs = data
            outputs = model(inputs)
            loss = criterion(outputs, inputs)
            val_epoch_loss += loss.item()

    # Print training and validation losses
    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Train Loss: {train_epoch_loss / len(train_dataloader):.4f}, '
          f'Val Loss: {val_epoch_loss / len(val_dataloader):.4f}')

Epoch [1/10000], Train Loss: 14912.9443, Val Loss: 14377.9268
Epoch [2/10000], Train Loss: 14686.3066, Val Loss: 14175.2051
Epoch [3/10000], Train Loss: 14474.3369, Val Loss: 13991.5322
Epoch [4/10000], Train Loss: 14284.4258, Val Loss: 13814.1494
Epoch [5/10000], Train Loss: 14103.7334, Val Loss: 13643.5449
Epoch [6/10000], Train Loss: 13930.7627, Val Loss: 13476.0117
Epoch [7/10000], Train Loss: 13760.7568, Val Loss: 13308.6035
Epoch [8/10000], Train Loss: 13591.4326, Val Loss: 13141.3760
Epoch [9/10000], Train Loss: 13423.3838, Val Loss: 12975.0078
Epoch [10/10000], Train Loss: 13256.2520, Val Loss: 12809.1787
Epoch [11/10000], Train Loss: 13089.0225, Val Loss: 12643.5010
Epoch [12/10000], Train Loss: 12922.0811, Val Loss: 12477.4424
Epoch [13/10000], Train Loss: 12755.2627, Val Loss: 12310.7041
Epoch [14/10000], Train Loss: 12588.1729, Val Loss: 12143.0381
Epoch [15/10000], Train Loss: 12420.4961, Val Loss: 11974.2549
Epoch [16/10000], Train Loss: 12251.9756, Val Loss: 11804.1934
E

In [7]:
# Save model
torch.save(model.state_dict(), '/Users/defeee/Documents/GitHub/FormAI-ML/Models/Core/Plank/plank_cnn.pth')